## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-01-13-39 +0000,nyt,Flight Delays Begin as Air Traffic Staffing Sh...,https://www.nytimes.com/2025/10/06/us/politics...
1,2025-10-07-01-12-33 +0000,nyt,Israel and Hamas Meet in Egypt to Discuss Gaza...,https://www.nytimes.com/2025/10/06/world/middl...
2,2025-10-07-01-11-37 +0000,nyt,Trump Said He Could Invoke the Insurrection Ac...,https://www.nytimes.com/2025/10/06/us/trump-in...
3,2025-10-07-01-00-00 +0000,wsj,"After Two Years of War, Israel Is Stronger—and...",https://www.wsj.com/world/middle-east/after-tw...
4,2025-10-07-00-51-09 +0000,cbc,"Sudan's civil war has faded from headlines, bu...",https://www.cbc.ca/news/world/el-fasher-darfur...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,52
312,new,24
86,government,15
87,shutdown,14
330,judge,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
28,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,122
18,2025-10-06-23-28-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,119
51,2025-10-06-22-14-52 +0000,latimes,Trump slams judge he picked as court tests lim...,https://www.latimes.com/california/story/2025-...,118
273,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...,115
116,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
28,122,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
229,83,2025-10-06-12-42-08 +0000,nypost,Bari Weiss to run new debate show at CBS News ...,https://nypost.com/2025/10/06/media/bari-weiss...
116,60,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
167,51,2025-10-06-17-27-02 +0000,bbc,Israel and Hamas begin indirect talks in Egypt...,https://www.bbc.com/news/articles/c708l7vgwywo...
51,44,2025-10-06-22-14-52 +0000,latimes,Trump slams judge he picked as court tests lim...,https://www.latimes.com/california/story/2025-...
178,36,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...
26,35,2025-10-06-23-00-36 +0000,nypost,Hundreds of Mount Everest hikers rescued after...,https://nypost.com/2025/10/06/world-news/hundr...
281,35,2025-10-06-07-36-00 +0000,wsj,Japan’s Next Prime Minister Could Make Rate In...,https://www.wsj.com/articles/japans-next-prime...
132,32,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
172,32,2025-10-06-17-19-13 +0000,nypost,White House blasts ‘SICK’ Chicago Mayor Brando...,https://nypost.com/2025/10/06/us-news/white-ho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
